In [13]:
import re, os, string, warnings, json
warnings.filterwarnings("ignore")
import math
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt', quiet=True)
from collections import Counter
from rapidfuzz import fuzz
from word2number import w2n
import dateparser
from dateutil import parser as duparser

In [3]:
df = pd.read_csv("synthetic_dataset.csv", encoding_errors="replace")

In [4]:
# This is being done so as to confirm to a same schema for data throughout the project
# Heuristic rename if your headers differ slightly
rename_map = {
    "reference": "gold_reference",
    "gold": "gold_reference",
    "answer": "ai_response",
    "response": "ai_response",
}
for k,v in rename_map.items():
    if k in df.columns and v not in df.columns:
        df.rename(columns={k:v}, inplace=True)

# Ensure exactly the 4 we use exist
for c in ["question","context","gold_reference","ai_response"]:
    if c not in df.columns: df[c] = ""

def _clean(s): 
    return (str(s) if pd.notna(s) else "").strip()

UNANSWERABLE_MARKERS = {
    "unanswerable","not answerable","cannot be determined","insufficient context",
    "not available in passage","unknown","na","n/a"
}

def derive_flags(row):
    gold = _clean(row["gold_reference"]).lower()
    ctx  = _clean(row["context"])
    is_unanswerable = gold in UNANSWERABLE_MARKERS
    context_missing = (ctx == "")
    no_gold = (gold == "")  # empty gold means judge vs context only
    return pd.Series({
        "is_unanswerable": is_unanswerable,
        "context_missing": context_missing,
        "no_gold": no_gold
    })

flags = df.apply(derive_flags, axis=1)
df = pd.concat([df, flags], axis=1)
df.head(3)

question  \
0  What did the Court of Justice not acknowledge?   
1  What did the Court of Justice not acknowledge?   
2  What did the Court of Justice not acknowledge?   

                                             context gold_reference  \
0  Fourth, national courts have a duty to interpr...            NaN   
1  Fourth, national courts have a duty to interpr...            NaN   
2  Fourth, national courts have a duty to interpr...            NaN   

                                         ai_response  is_unanswerable  \
0                    The passage doesn’t state this.            False   
1  It states that what did the court of justice w...            False   
2  The decision was influenced by the Paris Conve...            False   

   context_missing  no_gold  
0            False     True  
1            False     True  
2            False     True

In [ ]:
PROMPT_COL, RESPONSE_COL = "question", "ai_response"
assert PROMPT_COL in df.columns and RESPONSE_COL in df.columns

# ---------- custom functions for various use cases ----------
WORD_RX = re.compile(r"[A-Za-zÀ-ÖØ-öø-ÿ\u0900-\u097F0-9]+(?:[-’'][A-Za-z0-9]+)?")

def word_count(t: str) -> int:
    return len(WORD_RX.findall(t or ""))

def has_format_instruction(p: str):
    """
    Detect only the two format intents we care about: bulleted or paragraph.
    (We keep it minimal per your dataset.)
    """
    p = (p or "").lower()
    want_bullets   = any(k in p for k in ["bulleted", "bullet points", "as bullets", "list of", "bullets"])
    want_paragraph = "paragraph" in p
    return want_bullets, want_paragraph

def has_bullets(text: str) -> bool:
    """
    CUSTOM RULE (your dataset):
    A response is considered bulleted ONLY if it contains 'â€¢' or the literal word 'bullet'.
    No other symbols (e.g., '-', '*', '•') are recognized here.
    """
    if not text:
        return False
    t = text.lower()
    return ("â€¢" in t) or ("bullet" in t)

def follows_format(resp: str, want_bullets: bool, want_paragraph: bool):
    """
    Apply your strict bullet rule. 'paragraph' means text exists but has NO bullet markers.
    If neither format is requested, return True (no penalty for format).
    """
    if not (want_bullets or want_paragraph):
        return True
    is_bulleted  = has_bullets(resp)
    is_paragraph = (not is_bulleted) and bool((resp or "").strip())
    if want_bullets:   return is_bulleted
    if want_paragraph: return is_paragraph
    return True

def parse_exact_word_target(p: str):
    # exact N words
    m = re.search(r"(?:exact(?:ly)?)\s*(\d{1,4})\s+words?", (p or "").lower())
    return int(m.group(1)) if m else None

def parse_summarize_expand(p: str):
    """
    Detect summarize/expand intent and optional word limit for that intent.
    Supports 'summarize'/'summarise' variants.
    """
    pl = (p or "").lower()
    intent = None
    if any(x in pl for x in ["summarize","summarise","summary","tl;dr","tl dr","condense","shorten","brief"]):
        intent = "summarize"
    if any(x in pl for x in ["expand","elaborate","detail more","go deeper","make it longer"]):
        intent = "expand"
    m = re.search(r"(?:about|around|~)?\s*(\d{1,4})\s+words?", pl)
    wlim = int(m.group(1)) if m else None
    return intent, wlim

def band_penalty(measured: int, target: int, band: int, cap: float):
    """
    Linear penalty after grace band; returns 0..cap
    """
    if target is None:
        return 0.0
    dev = abs(measured - target)
    if dev <= band:
        return 0.0
    return min(cap, (dev - band) / max(1, target) * cap * 2)

def has_any_instruction(p: str) -> bool:
    pl = (p or "").lower()
    return any(k in pl for k in [
        "bullet", "paragraph", "exact", "words",
        "summarize", "summarise", "expand", "tl;dr", "tl dr", "brief", "shorten"
    ])



In [ ]:
# ---------- scoring (only 4 cases) ----------
# 1) Format (bulleted/paragraph) using your strict bullet markers.
# 2) Exact N words with ±1 grace.
# 3) If no instruction at all -> 1.0.
# 4) Summarize/Expand with optional word limit and ±(5–10) grace band (default 7).

# designing is such that --> higher instruction score == better instruction following 

# Keep weights simple
FORMAT_W = 0.4
LENGTH_W = 0.6

def instruction_score_minimal(prompt: str, resp: str, summarize_expand_band: int = 7):
    if not has_any_instruction(prompt):
        return 1.0, ["no_instruction"], ["full credit (no instruction)"]

    flags, notes = [], []
    score_format = 1.0
    score_length = 1.0

    # (1) format
    want_bullets, want_paragraph = has_format_instruction(prompt)
    if want_bullets or want_paragraph:
        ok = follows_format(resp, want_bullets, want_paragraph)
        score_format = 1.0 if ok else 0.3
        if not ok:
            flags.append("format_mismatch")
        notes.append(f"format:{score_format:.2f}")

    # (2) exact words 
    exact_target = parse_exact_word_target(prompt)
    intent, se_limit = parse_summarize_expand(prompt)
    if intent:
        flags.append(f"intent:{intent}")

    w = word_count(resp)

    if exact_target is not None:
        pen = band_penalty(w, exact_target, band=1, cap=0.7) 
        if pen > 0:
            flags.append(f"exact_words_off:{w}/{exact_target}")
        score_length = max(0.0, 1.0 - pen)
        notes.append(f"exact_len:{score_length:.2f}")
    elif se_limit is not None:
        pen = band_penalty(w, se_limit, band=summarize_expand_band, cap=0.7)  
        if pen > 0:
            flags.append(f"se_words_off:{w}/{se_limit}")
        score_length = max(0.0, 1.0 - pen)
        notes.append(f"se_len:{score_length:.2f}")
    else:
        notes.append("len:n/a")


    final = FORMAT_W * score_format + LENGTH_W * score_length
    return round(max(0.0, min(1.0, final)), 4), flags, notes

In [8]:
# ---------- applying evaluation metrics to dataframe ----------
scores, flags, notes = [], [], []
for pr, rs in zip(df[PROMPT_COL].astype(str), df[RESPONSE_COL].astype(str)):
    s, f, n = instruction_score_minimal(pr, rs, summarize_expand_band=7)  
    scores.append(s); flags.append(";".join(f)); notes.append(";".join(n))

df["instr_score"] = scores
df["instr_flags"] = flags
df["instr_notes"] = notes

In [11]:
print(df[[PROMPT_COL, RESPONSE_COL, "instr_score", "instr_flags"]].head(8).to_string(index=False))

                                              question                                                                                                                             ai_response  instr_score    instr_flags
        What did the Court of Justice not acknowledge?                                                                                                         The passage doesn’t state this.          1.0 no_instruction
        What did the Court of Justice not acknowledge?                                   It states that what did the court of justice was fully acknowledged, including details not mentioned.          1.0 no_instruction
        What did the Court of Justice not acknowledge? The decision was influenced by the Paris Convention Committee, which clarified the issue, though the passage doesn’t mention this body.          1.0 no_instruction
        What did the Court of Justice not acknowledge?                                                                      

In [12]:
df.to_csv("scored datasets/instruction_scores_added.csv")